# RightSignature Certificate Scanner

In [1]:
import sys
import os
import re
from PyPDF2 import PdfReader
from openpyxl import Workbook
from openpyxl.styles import Color, colors, PatternFill

!{sys.executable} -m pip install PyPDF2
!{sys.executable} -m pip install openpyxl

In [2]:
#INSERT YOUR OWN FOLDER PATH HERE
#Folder should consist of ONLY the PDFs that need to be checked
pathName = r"C:\Users\dkuang\Downloads\Davids_Folder"
os.chdir(pathName)

## ref_finder() takes the name of the file and its page length (w/o certificate) and returns a reference number if found

### It's possible to also extract the names of the Bigs and Littles to add to a spreadsheet- let me know if you'd like that

In [4]:
def ref_finder(fileName, doclength):
    file_pdf = PdfReader(fileName)
    num_pages = len(file_pdf.pages)
    if (num_pages <= doclength): #if there are no additional pages,
        return "" #then there certainly isn't a rightSignature certificate
    else:
        cert_page = file_pdf.pages[doclength] #isolates first page of certificate
        cert_content = cert_page.extract_text()
        #searches certificate page for a potential reference number
        ref_num = re.search(r"\w{8}-\w{4}-\w{4}-\w{4}-\w{12}", cert_content)
        if (ref_num == None):
            return ""
        else:
            return ref_num.group()

#test
print(ref_finder("69374.pdf", 2))

## folder_refs() scans through a folder of PDFs and returns a dictionary of file names with reference numbers

In [5]:
def folder_refs(path):
    files = os.listdir(path)
    ref_dict = dict()
    for item in files:
        ref_dict[item] = ref_finder(item, 2)
        #for common ground agreements, page length is 2
    return ref_dict

#test
print(folder_refs(pathName))

{'69374.pdf': '', '69380.pdf': '', '69390.pdf': '', '69391.pdf': '53B7EE86-6D80-4983-8C05-C7EC579A8F32', '69416.pdf': '', '69422.pdf': '', '69424.pdf': '', '69426.pdf': '', '69428.pdf': '', '69475.pdf': '', '69520.pdf': 'DCFA6CAB-D71E-4FAA-A5AD-9664A445FA3A', '69594.pdf': '', '69595.pdf': '', '70152.pdf': 'D944484B-01B1-42A6-BE67-7710580E0BB2', '70284.pdf': 'D1E49C4C-5C82-4654-B0D9-BE1E1AA1E459', '70602.pdf': 'BC6BED7D-CFB3-40BD-8C73-BB834718722C', '70673.pdf': '68A0512C-8ACD-438E-BD5C-66038380847F', '70682.pdf': '66EB1B64-BE67-44CD-9719-12B779398E9D', '70784.pdf': 'B2D314DC-3AEE-43CC-8753-71176CBE94E1', '70969.pdf': 'E7F100C5-8401-487A-ACC9-C9A7635705FC', '71063.pdf': 'DACF48B2-232C-408C-B3DB-E74344559644', '71070.pdf': 'BA88A5D0-74F5-43DC-AC56-C86C23B8AB36'}


### Here's the magic: outputting an excel file with the Match IDs and their RightSignature reference numbers. If no certificate is found, they are highlighted in red. 

In [6]:
wb = Workbook()
ws = wb[wb.sheetnames[0]]

titles = ws.cell(row = 1, column = 1, value = "File Name")
references = ws.cell(row = 1, column = 2, value = "RightSignature Reference Number")
ws.freeze_panes = ws["A2"]

results = folder_refs(pathName).items()

i = 2
for file, ref in results:
    file_cell = ws.cell(row = i, column = 1, value = file)
    ref_cell = ws.cell(row = i, column = 2, value = ref)
    if (ref.__eq__("")):
        file_cell.fill = PatternFill("solid", fgColor="FF0000")
        ref_cell.fill = PatternFill("solid", fgColor="FF0000")
    i += 1
    
#to store the excel file in another folder:
#os.chdir(r"path...")
        
wb.save("tester.xlsx")
